In [1]:
import yfinance as yf
import finplot as fplt
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf

%reload_ext autoreload
%autoreload 2

In [81]:
msft = yf.Ticker("MSFT")

# get all stock info
msft.info

# get historical market data
hist = msft.history(period="70y", interval="1d")
# hist.to_csv("MSFT.csv")
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 00:00:00-05:00,0.055004,0.063093,0.055004,0.060396,1031788800,0.0,0.0
1986-03-14 00:00:00-05:00,0.060396,0.063632,0.060396,0.062553,308160000,0.0,0.0
1986-03-17 00:00:00-05:00,0.062553,0.064172,0.062553,0.063632,133171200,0.0,0.0
1986-03-18 00:00:00-05:00,0.063632,0.064172,0.061475,0.062014,67766400,0.0,0.0
1986-03-19 00:00:00-05:00,0.062014,0.062553,0.060396,0.060936,47894400,0.0,0.0


In [ ]:
from utils import DenseLine as CFig


targets, predicts = [], []

n, nplot, wait = 1, 0, 0
fig = mpf.figure(figsize=(20, 8))
for t in range(100, 9000):
    if wait:
        wait -= 1
        continue
    candlefig = CFig(20, 5, 3, 1)
    if candlefig.update(hist, t):
        targets.append(candlefig.target)
        predicts.append(candlefig.prediction)
        wait = candlefig.target_length
        if nplot < 8:
            ax = fig.add_subplot(1,2,n) # main candle stick chart subplot, you can also pass in the self defined style here only for this subplot
            n += 1
            if n > 2:
                n = 1
                fig = mpf.figure(figsize=(20, 8))
            mpf.plot(hist[t-candlefig.body_length-candlefig.trend_length:t+candlefig.target_length+1], type='candle', ax=ax, alines=candlefig.lines)
            nplot += 1
        
print(len(targets))

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
targets = np.array(targets)
predicts = np.array(predicts)
print(classification_report(targets>0, predicts>0))
res = (targets * predicts)
acc = ((targets>0) == (predicts>0)).sum()/targets.shape[0]
print(res.sum()/len(targets))
plt.figure(figsize=(20, 7))
plt.subplot(1, 2, 1)
plt.plot(res);
plt.subplot(1, 2, 2)
plt.plot(res.cumsum())
plt.plot(targets.cumsum(), "--")
plt.legend(["model", "buy&hold"]);

In [52]:
import pickle
from pathlib import Path
from easydict import EasyDict
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


cfgs, btests = [], []
for p in Path("optimization").glob("*.pickle"):
    cfg, btest = pickle.load(open(p, "rb"))
    cfgs.append(cfg)
    btests.append(btest)

In [60]:
opt_summary = {k:[] for k in cfgs[0].keys()}
for k in opt_summary.keys():
    for cfg in cfgs:
        v = cfg[k]
        if type(v) is EasyDict and "func" in v.keys():
            opt_summary[k].append(str(v.func.name))
        else:
            opt_summary[k].append(v)
            
opt_summary["btest"] = []
for btest in btests:
    opt_summary["btest"].append(btest.profits.sum())
    
opt_summary = pd.DataFrame(opt_summary)
opt_summary.sort_values(by=["btest"], ascending=False, inplace=True)
opt_summary.head(10)

,body_classifier,stops_processor,wait_entry_point,hist_buffer_size,tstart,tend,period,ticker,data_type,save_plots,btest
49,trend:npairs=2,"stops_fix:tp=4,sl=2",9999,30,0,None,H1,TRXUSD,bitfinex,False,233.563815
18,trend:npairs=3,"stops_fix:tp=4,sl=2",9999,30,0,None,H1,TRXUSD,bitfinex,False,209.957105
25,trend:npairs=2,"stops_fix:tp=4,sl=4",9999,30,0,None,H1,TRXUSD,bitfinex,False,209.010223
37,trend:npairs=3,stops_dyn:dummy=0,9999,30,0,None,H1,BTCUSD,bitfinex,False,191.404104
47,trend:npairs=2,"stops_fix:tp=2,sl=2",9999,30,0,None,H1,ETHUSD,bitfinex,False,184.000000
78,trend:npairs=3,"stops_fix:tp=4,sl=2",9999,30,0,None,H1,BTCUSD,bitfinex,False,146.000000
66,trend:npairs=3,"stops_fix:tp=4,sl=4",9999,30,0,None,H1,TRXUSD,bitfinex,False,139.982140
72,trend:npairs=2,stops_dyn:dummy=0,9999,30,0,None,H1,BTCUSD,bitfinex,False,118.645726
38,trend:npairs=2,"stops_fix:tp=2,sl=2",9999,30,0,None,H1,TRXUSD,bitfinex,False,103.222939
20,trngl_simp:npairs=3,"stops_fix:tp=4,sl=4",9999,30,0,None,H1,TRXUSD,bitfinex,False,80.048780


In [58]:
opt_res = {"expert":[], "ticker":[], "btest":[]}
for i in range(opt_summary.shape[0]):
    exphash = ""
    for col in opt_summary.columns:
        if col != "ticker" and col != "btest":
            exphash += str(opt_summary[col].iloc[i]) + " "
    opt_res["expert"].append(exphash)
    opt_res["ticker"].append(opt_summary.ticker.iloc[i])
    opt_res["btest"].append(opt_summary.btest.iloc[i])

opt_res = pd.DataFrame(opt_res)
opt_res = opt_res.groupby(by="expert").sum()
opt_res["btest_std"] = opt_res.groupby(by="expert").std(numeric_only=True)["btest"]

opt_res.sort_values(by=["btest"], ascending=False, inplace=True)

opt_res    

,ticker,btest,btest_std
expert,,,
"trend:npairs=3 stops_fix:tp=4,sl=2 9999 30 0 None H1 bitfinex False",TRXUSDBTCUSDXRPUSDETHUSD,433.829724,NaN
"trend:npairs=2 stops_fix:tp=4,sl=2 9999 30 0 None H1 bitfinex False",TRXUSDETHUSDXRPUSDBTCUSD,394.741024,NaN
"trend:npairs=2 stops_fix:tp=2,sl=2 9999 30 0 None H1 bitfinex False",ETHUSDTRXUSDXRPUSDBTCUSD,354.900347,NaN
"trend:npairs=2 stops_fix:tp=4,sl=4 9999 30 0 None H1 bitfinex False",TRXUSDXRPUSDETHUSDBTCUSD,284.882843,NaN
"trend:npairs=2 stops_fix:tp=2,sl=4 9999 30 0 None H1 bitfinex False",TRXUSDETHUSDBTCUSDXRPUSD,200.846065,NaN
"trend:npairs=3 stops_fix:tp=2,sl=2 9999 30 0 None H1 bitfinex False",BTCUSDTRXUSDETHUSDXRPUSD,168.670911,NaN
trend:npairs=2 stops_dyn:dummy=0 9999 30 0 None H1 bitfinex False,BTCUSDXRPUSDETHUSDTRXUSD,166.222361,NaN
"trend:npairs=3 stops_fix:tp=4,sl=4 9999 30 0 None H1 bitfinex False",TRXUSDBTCUSDETHUSDXRPUSD,59.854760,NaN
"trngl_simp:npairs=3 stops_fix:tp=4,sl=2 9999 30 0 None H1 bitfinex False",TRXUSDETHUSDBTCUSDXRPUSD,40.063057,NaN


In [70]:
opt_res = {"expert":[], "ticker":[], "btest":[]}
for i in range(opt_summary.shape[0]):
    exphash = ""
    for col in opt_summary.columns:
        if col != "ticker" and col != "btest":
            exphash += str(opt_summary[col].iloc[i]) + " "
    opt_res["expert"].append(exphash)
    opt_res["ticker"].append(opt_summary.ticker.iloc[i])
    opt_res["btest"].append(opt_summary.btest.iloc[i])

opt_res = pd.DataFrame(opt_res)
opt_res = opt_res.groupby(by="expert").sum()
opt_res.groupby(by="expert").std(numeric_only=True)

,btest
expert,
trend:npairs=2 stops_dyn:dummy=0 9999 30 0 None H1 bitfinex False,NaN
"trend:npairs=2 stops_fix:tp=2,sl=2 9999 30 0 None H1 bitfinex False",NaN
"trend:npairs=2 stops_fix:tp=2,sl=4 9999 30 0 None H1 bitfinex False",NaN
"trend:npairs=2 stops_fix:tp=4,sl=2 9999 30 0 None H1 bitfinex False",NaN
"trend:npairs=2 stops_fix:tp=4,sl=4 9999 30 0 None H1 bitfinex False",NaN
trend:npairs=3 stops_dyn:dummy=0 9999 30 0 None H1 bitfinex False,NaN
"trend:npairs=3 stops_fix:tp=2,sl=2 9999 30 0 None H1 bitfinex False",NaN
"trend:npairs=3 stops_fix:tp=2,sl=4 9999 30 0 None H1 bitfinex False",NaN
"trend:npairs=3 stops_fix:tp=4,sl=2 9999 30 0 None H1 bitfinex False",NaN
